In [ ]:
# if super busted, shut down kernel, remove ~/.julia, start a REPL and reinstall IJulia, then restart here

In [ ]:
if isfile("Project.toml")
    println("removing Project.toml...")
    rm("Project.toml")
end
if isfile("Manifest.toml")
    println("removing Manifest.toml...")
    rm("Manifest.toml")
end
ENV["LD_LIBRARY_PATH"] = ""

import Pkg
Pkg.activate(".")
# Pkg.update()
# delete baseline environment???
# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
# import Mycelia

pkgs = [
"DataFrames",
"uCSV",
"ProgressMeter"
# "StatsPlots"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")
results_directory = joinpath(data_dir, "results")

In [ ]:
# blastdbs_dir = "$(homedir())/workspace/blastdbs"
# taxdump_dir = mkpath(joinpath(blastdbs_dir, "taxdump"))
# taxdump_tar = joinpath(taxdump_dir, "taxdump.tar.gz")
# if !isfile(taxdump_tar)
#     run(`wget --quiet https://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz --directory-prefix=$(taxdump_dir)`)
# end
# if isempty(filter(x -> occursin(r"\.dmp$", x), readdir(taxdump_dir)))
#     run(`tar -xvzf $(taxdump_tar) --directory $(taxdump_dir)`)
# end

In [ ]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)

# get all taxonids at or below virus
# mamba create -n taxonkit -c bioconda taxonkit
# wget -c ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz 
# tar -zxvf taxdump.tar.gz
# mkdir -p $HOME/.taxonkit
# cp names.dmp nodes.dmp delnodes.dmp merged.dmp $HOME/.taxonkit
# --data-dir
viral_tax_ids = Set(parse.(Int, filter(!isempty, readlines(`conda run -n taxonkit taxonkit list --ids 10239 --indent ""`))))
# n_methods = 8
data_dir = joinpath(dirname(pwd()), "data")
RESULTS_DIR = mkpath(joinpath(data_dir, "results"))

# SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))

In [ ]:
# ICTV_viral_metadata = 
# NCBI_viral_metadata = 
# VirusHostDB_viral_metadata = 

In [ ]:
joint_viral_hits_table_file = joinpath(RESULTS_DIR, "joint-viral-blast-hits.tsv")
# joint_viral_hits_table = DataFrames.DataFrame()
# ProgressMeter.@showprogress for SRR_path in SRR_paths
#     coverage_and_classification_reports = filter(x -> occursin("coverage-and-classification.tsv", x), readdir(joinpath(SRR_path, "blastn")))
#     @assert length(coverage_and_classification_reports) == 1
#     coverage_and_classification_report = first(coverage_and_classification_reports)
#     coverage_and_classification_report_path = joinpath(joinpath(SRR_path, "blastn", coverage_and_classification_report))
#     coverage_and_classification_table = DataFrames.DataFrame(uCSV.read(coverage_and_classification_report_path, delim='\t', header=1, types=String, allowmissing=true, encodings=Dict("" => missing)))
#     coverage_and_classification_table[!, "sample_id"] .= basename(SRR_path)
#     coverage_and_classification_table[!, "subject tax id"] = parse.(Int, coverage_and_classification_table[!, "subject tax id"])
#     coverage_and_classification_table = coverage_and_classification_table[map(x -> x in viral_tax_ids, coverage_and_classification_table[!, "subject tax id"]), :]
#     append!(joint_viral_hits_table, coverage_and_classification_table, promote=true, cols=:union)
# end
# uCSV.write(joint_viral_hits_table_file, ifelse.(ismissing.(joint_viral_hits_table), "", joint_viral_hits_table), delim='\t')

In [ ]:
joint_viral_hits_table_with_metadata_file = joinpath(RESULTS_DIR, "joint-viral-blast-hits.with-host-metadata.tsv")

# metadata_dir = joinpath(dirname(pwd()), "metadata")
# ncbi_virus_metadata_file = joinpath(metadata_dir, "NCBI-virus-refseq.transformed.tsv")
# ncbi_virus_metadata = DataFrames.DataFrame(uCSV.read(ncbi_virus_metadata_file, header=1, delim='\t'))
# joint_viral_hits_table_with_metadata = DataFrames.innerjoin(
#     joint_viral_hits_table,
#     unique(ncbi_virus_metadata[!, ["taxid", "host_is_vertebrate", "host_is_mammal", "host_is_primate", "host_is_human"]]),
#     on="subject tax id" => "taxid"
# )[!, DataFrames.Not("subject tax ids")]
# uCSV.write(joint_viral_hits_table_with_metadata_file, joint_viral_hits_table_with_metadata, delim='\t')

In [ ]:
# readdir(metadata_dir)
# "VMR_MSL38_v1 - VMR MSL38 v1.transformed.tsv"
# "NCBI-virus-refseq.transformed.tsv"
# "virushostdb.transformed.tsv"

In [ ]:
# contig_info_table
# blast_hits_top_hits_table
# joint_lca_table
# genomad_results
# virsorter_results

In [ ]:
p = StatsPlots.scatter(
    viral_blast_hits_joint_table[!, "alignment length"],
    viral_blast_hits_joint_table[!, "% identity"],
    title = "aligment quality for conensus viral contigs",
    xlabel = "alignment length",
    ylabel = "% identity",
    legend=false,
    size=(600, 400)
)

StatsPlots.savefig(p, joinpath(results_directory, "figure-1.$(now_string).png"))
StatsPlots.savefig(p, joinpath(results_directory, "figure-1.$(now_string).pdf"))
# StatsPlots.savefig(p, joinpath(RESULTS_DIR, "figure-1.$(now_string).jpeg"))
# display(p)

In [ ]:
viral_blast_hits_joint_table_file = last(filter(x -> occursin("figure-1.viral_blast_hits_joint_table", x), filter(x -> occursin(".tsv", x), readdir(results_directory, join=true))))

In [ ]:
viral_hits_joint_table = DataFrames.DataFrame(uCSV.read(viral_blast_hits_joint_table_file, delim='\t', header=true))

In [ ]:
# get top hits by length to annotate what we're seeing

In [ ]:
# assess total # of viral families

In [ ]:
# assess total # of viral genera

In [ ]:
# group the above by family and genera

In [ ]:
# StatsBase.describe(viral_blast_hits_joint_table[!, "alignment length"])

In [ ]:
# viral_blast_hits_joint_table[viral_blast_hits_joint_table[!, "alignment length"] .>= 100, :]

In [ ]:
# taxdump_dir

In [ ]:
taxid = 9606

In [ ]:
taxid, ranks, taxids, split(strip(read(pipeline(`echo $taxid`, `conda run --no-capture-output -n taxonkit taxonkit --data-dir $(taxdump_dir) lineage --show-lineage-ranks --show-lineage-taxids --show-rank`), String)), '\t')

In [ ]:
conda run --no-capture-output -n taxonkit taxonkit --data-dir $HOME/workspace/blastdbs/taxdump list --indent "" --ids 1 > taxids.txt


conda run --no-capture-output -n taxonkit taxonkit --data-dir $HOME/workspace/blastdbs/taxdump lineage --show-rank taxids.txt > taxon-lineages.txt

In [ ]:
# function taxid_to_ranked_lineage(taxid)

    taxid_lineage_reformatted = join(split(taxid_lineage, ';'), '\n')
    d = Dict()
    for line in readlines(pipeline(`echo $(taxid_lineage_reformatted)`, `conda run --no-capture-output -n taxonkit taxonkit --data-dir $(taxdump_dir) name2taxid --show-rank`))
        rank_name, rank_taxid, rank = string.(split(line, '\t'))
        d[rank] = Dict("name" => rank_name, "taxid" => parse(Int, rank_taxid))
    end
    return d
# end

In [ ]:
# # conda create -n taxonkit -c bioconda taxonkit
# taxid = 353768
# )

In [ ]:
taxon_lineages = Dict()
ProgressMeter.@showprogress for taxid in unique(viral_blast_hits_joint_table[!, "subject tax id"])
    taxon_lineages[taxid] = taxid_to_ranked_lineage(taxid)
end

In [ ]:
# taxonkit list --ids 1 --indent "" | taxonkit lineage --show-lineage-ranks --show-lineage-taxids > t
# taxonkit list --ids 1 --indent "" 




echo 9606 | conda run --no-capture-output -n taxonkit taxonkit --data-dir $HOME/workspace/blastdbs/taxdump lineage --show-lineage-ranks --show-lineage-taxids --show-rank

In [ ]:
  -R, --show-lineage-ranks    appending ranks of all levels
  -t, --show-lineage-taxids   appending lineage consisting of taxids
  -n, --show-name             appending scientific name
  -r, --show-rank             appending rank of taxids

In [ ]:
# contig_info_table[!, "viral_classification_count"] = map(contig -> get(contig_support_counts, contig, 0), contig_info_table[!, "Contig"])
# contig_info_table[!, "viral_classification_percent"] = round.((contig_info_table[!, "viral_classification_count"] ./ n_methods) .* 100, digits=1)
# end
# sample_summary_table

In [ ]:
# SRR_paths
# SRR_path = rand(SRR_paths)

In [ ]:
# IMGVR_sam = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.viral.fna.IMGVR_all_nucleotides-high_confidence.fna.gz.sam")

In [ ]:
# uCSV.read(IMGVR_sam, delim='\t', typedetectrows=100)

In [ ]:
# reader = open(XAM.SAM.Reader, IMGVR_sam)
# # Iterate over BAM records.
# for record in reader
#     # `record` is a BAM.Record object.
#     if XAM.SAM.ismapped(record)
#         # Print the mapped position.
#         println(XAM.SAM.refname(record), ':', XAM.SAM.position(record))
#     end
# end

# # Close the BAM file.
# close(reader)

Based on these results, we will go for a 3x confirmation via targetted blast where the sequence must have a high confidence match for a viral hit using:
- ref_viruses_rep_genomes_blastn
- ref_viruses_rep_genomes_dcmegablast
- nt_viral_validation_megablast

Full nt database attempts to validate viral contigs came back primarily with bacterial artificial chromosomes associated with human cell lines that did not seem like valuable hits to us. We also didn't expect to find *novel* viruses by megablasting against the ref_viruses_rep_genomes, given that it is a limited, representative set of all known viruses and the official description of the algorithm is "Traditional megablast used to find very similar (e.g., intraspecies or closely related species) sequences"
https://www.ncbi.nlm.nih.gov/books/NBK569839/

Because of this, we felt that getting a multiple redudant hits using the most flexibile algorithm (blastn) against the highest quality blast db (ref_viruses_rep_genomes), a less flexible, but still cross-species algorithm (dc-megablast: Discontiguous megablast used to find more distant (e.g., interspecies) sequences), and finally a 3rd validation hit against a potentially lower quality due to less manual curation viral database (nt_viral) using the strictest alogrithm, megablast.

Because of the low concordance of Kraken classifications to blast classifications, we consider the calls with much less weight than the classified contigs, but include them because the data was generated and it may prove informative to others.

As an update to the above, we found that we were unable to perform the `nt_viral_validation_megablast` in a reasonable amount of time, so we will be dropping that requirement.

Instead, we will perform a final re-assembly and then perform the final validation blast and protein-level classification on that final assembly

The following was a manual, indepth analysis of the first sample that we used to inform our validation approach

In [ ]:
# # extract contigs that came back as viral in the targetted screen
# # blast back against NCBI nt and confirm they are still viral
# SRR_path = first(SRR_paths)
# SRR = basename(SRR_path)

# # ref_viruses_ref_genomes_blast_report = joinpath(SRR_path, "blastn", "final.contigs.fastg.gfa.fna.blastn.ref_viruses_rep_genomes.blastn.txt")
# # ref_viruses_ref_genomes_blast_results = Mycelia.parse_blast_report(ref_viruses_ref_genomes_blast_report)
# # possible_viral_contigs = Set(unique(ref_viruses_ref_genomes_blast_results[!, "query id"]))
# # assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")
# # viral_fasta = replace(assembled_fasta, ".fna" => ".potential_viral_contigs.fna")
# # potential_viral_records = filter(x -> FASTX.identifier(x) in possible_viral_contigs, collect(Mycelia.open_fastx(assembled_fasta)))
# # open(viral_fasta, "w") do io
# #     fastx_io = FASTX.FASTA.Writer(io)
# #     for record in potential_viral_records
# #         write(fastx_io, record)
# #     end
# #     close(fastx_io)
# # end

# # potential_viral_records


# taxon_id_to_kingdom_map = Dict{Int, String}()
# kingdom_to_taxon_id_map = Dict(
#     "Viruses" => 10239,
#     "Archaea" => 2157,
#     "Bacteria" => 2,
#     "Eukaryota" => 2759,
#     "Other" => 28384,
#     "Unclassified" => 12908
# )
# ProgressMeter.@showprogress for (kingdom, taxon_id) in kingdom_to_taxon_id_map
#     for child_taxon_id in parse.(Int, filter(!isempty, readlines(`taxonkit list --data-dir $(taxdump_dir) --ids $(taxon_id) --indent=""`)))
#         taxon_id_to_kingdom_map[child_taxon_id] = kingdom
#     end
# end

# ref_viruses_rep_genomes_blastn_contigs = unique(ref_viruses_rep_genomes_blastn_results[!, "query id"])

# ref_viruses_rep_genomes_dcmegablast_contigs = unique(ref_viruses_rep_genomes_dcmegablast_results[!, "query id"])

# # # 8 non-overlapping hits!
# # union(ref_viruses_rep_genomes_dcmegablast_contigs, ref_viruses_rep_genomes_blastn_contigs)
# # # these all have pretty low e-values, I'm not going to worry about them
# # dcmegablast_only = setdiff(ref_viruses_rep_genomes_dcmegablast_contigs, ref_viruses_rep_genomes_blastn_contigs)
# # ref_viruses_rep_genomes_dcmegablast_results[map(x -> x in dcmegablast_only, ref_viruses_rep_genomes_dcmegablast_results[!, "query id"]), :]

# ref_viruses_rep_genomes_megablast_contigs = unique(ref_viruses_rep_genomes_megablast_results[!, "query id"])
# # nt_viral_validation_megablast_results
# # nt_validation_megablast_results
# # all megablast hits are subset of blastn hits
# # union(ref_viruses_rep_genomes_megablast_contigs, ref_viruses_rep_genomes_blastn_contigs)

# nt_viral_validation_megablast_contigs = unique(nt_viral_validation_megablast_results[!, "query id"])
# # nt_validation_megablast_results

# # union(nt_viral_validation_megablast_contigs, ref_viruses_rep_genomes_blastn_contigs)
# # map(x -> x in viral_taxon_ids, nt_viral_validation_megablast_results[!, "subject tax id"])

# taxon_id_to_kingdom_map

# # only 12 contigs are still considered viral contigs after mapping to nt complete, but the hits aren't very convicing (bacterial artificial chromosomes?)
# nt_validation_megablast_contigs = unique(nt_validation_megablast_results[map(x -> get(taxon_id_to_kingdom_map, x, "") == "Viruses", nt_validation_megablast_results[!, "subject tax id"]), "query id"])

# full_contig_set = union(nt_validation_megablast_contigs, nt_viral_validation_megablast_contigs, ref_viruses_rep_genomes_blastn_contigs, ref_viruses_rep_genomes_dcmegablast_contigs, ref_viruses_rep_genomes_megablast_contigs)

# # get all of the reads mapping to each

# bam_file = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna.bwa.bam")
# # bamfile = first(filter(x -> occursin(r"\.bam$", x), readdir(joinpath(SRR_dir, "megahit"), join=true)))

# # implement as the following nested dictionary
# # contigs => reads => taxon_id

# function generate_contig_to_reads_map(bamfile, contigs_of_interest)
#     contigs_to_reads_map = Dict(contig => Set{String}() for contig in contigs_of_interest)
#     # reads_of_interest = Set{String}()
#     reader = open(XAM.BAM.Reader, bamfile)
#     for record in reader
#         if XAM.BAM.ismapped(record) && (XAM.BAM.refname(record) in contigs_of_interest)
#             push!(contigs_to_reads_map[XAM.BAM.refname(record)], XAM.BAM.tempname(record))
#         end
#     end
#     close(reader)
#     return contigs_to_reads_map
# end

# # 1200 seconds
# @time contigs_to_reads_map = generate_contig_to_reads_map(bam_file, full_contig_set)

# function read_kraken_output(kraken_output)
#     # read_kraken_report
#     header = [
#         "classification status",
#         "sequence ID",
#         "taxon ID",
#         "sequence length",
#         "LCA mappings"
#     ]
#     data, _ = uCSV.read(IOBuffer(join(filtered_lines, '\n')), delim='\t')
#     return DataFrames.DataFrame(data, header)
# end

# # readdir(joinpath(SRR_path, "kraken"))

# reads_of_interest = reduce(union, values(contigs_to_reads_map))

# kraken_output = last(filter(x -> occursin(r"\.kraken-output\.tsv", x), readdir(joinpath(SRR_path, "kraken"), join=true)))
# if occursin(r"\.gz", kraken_output)
#     kraken_buffer = open(`gzip -dc $(kraken_output)`)
# else
#     kraken_buffer = open(kraken_output)
# end

# filtered_lines = String[]
# for line in eachline(kraken_buffer)
#     split_line = split(line, '\t')
#     if split_line[2] in reads_of_interest
#         push!(filtered_lines, line)
#     end
# end
# close(kraken_buffer)

# read_classifications = read_kraken_output(IOBuffer(join(filtered_lines, '\n')))
# read_classifications[!, "parsed taxon ID"] = map(x -> match(r"\(taxid (\d+)\)", x).captures[1], read_classifications[!, "taxon ID"])
# read_classifications

# read_classifications_map = Dict(row["sequence ID"] => parse(Int, row["parsed taxon ID"]) for row in DataFrames.eachrow(read_classifications))

# contigs_to_taxon_counts_map = Dict()
# for (contig, reads) in contigs_to_reads_map
#     contigs_to_taxon_counts_map[contig] = StatsBase.countmap(get(taxon_id_to_kingdom_map, read_classifications_map[read], "Unclassified") for read in reads)
# end
# contigs_to_taxon_counts_map

# contigs_to_taxon_proportions_map = Dict()
# for (contig, taxon_counts) in contigs_to_taxon_counts_map
#     total_count = sum(values(taxon_counts))
#     contigs_to_taxon_proportions_map[contig] = Dict(kingdom => count / total_count for (kingdom, count) in taxon_counts)
# end
# contigs_to_taxon_proportions_map

# # ref_viruses_rep_genomes_blastn_results
# # ref_viruses_rep_genomes_dcmegablast_results
# # ref_viruses_rep_genomes_megablast_results
# # nt_viral_validation_megablast_results
# # nt_validation_megablast_results

# contig_classification_results = 
# DataFrames.DataFrame(
#     union(
#         DataStructures.OrderedDict("Contig" => String[]),
#         DataStructures.OrderedDict(k => Float64[] for k in keys(kingdom_to_taxon_id_map)),
#         # these are ordered by most hits to fewest hits
#         DataStructures.OrderedDict(db_algorithm => Bool[] for db_algorithm in ["ref_viruses_rep_genomes_blastn", "ref_viruses_rep_genomes_dcmegablast", "nt_viral_validation_megablast", "ref_viruses_rep_genomes_megablast", "nt_validation_megablast"])
#     )
# )

# for (contig, taxon_proportions) in contigs_to_taxon_proportions_map
#     row = Dict{Any, Any}("Contig" => contig)
#     for k in keys(kingdom_to_taxon_id_map)
#         row[k] = get(taxon_proportions, k, 0.0)
#     end
#     row["ref_viruses_rep_genomes_blastn"] = contig in ref_viruses_rep_genomes_blastn_contigs
#     row["nt_viral_validation_megablast"] = contig in nt_viral_validation_megablast_contigs
#     row["ref_viruses_rep_genomes_dcmegablast"] = contig in ref_viruses_rep_genomes_dcmegablast_contigs
#     row["ref_viruses_rep_genomes_megablast"] = contig in ref_viruses_rep_genomes_megablast_contigs
#     row["nt_validation_megablast"] = contig in nt_validation_megablast_contigs
#     push!(contig_classification_results, row)
# end
# contig_classification_results[!, "top_kingdom"] .= ""
# for (i, row) in enumerate(DataFrames.eachrow(contig_classification_results))
#     max_hit = ""
#     max_value = 0.0
#     for k in keys(kingdom_to_taxon_id_map)
#         if row[k] > max_value
#             max_value = row[k]
#             max_hit = k
#         end
#     end
#     contig_classification_results[i, "top_kingdom"] = max_hit
# end

# m = Int.(Matrix(
#     contig_classification_results[!, [
#         "ref_viruses_rep_genomes_blastn",
#         "ref_viruses_rep_genomes_dcmegablast",
#         "nt_viral_validation_megablast",
#         "ref_viruses_rep_genomes_megablast",
#         "nt_validation_megablast"
#     ]
# ]))

# contig_classification_results[!, "blast_hits"] = map(r -> sum(r), eachrow(m))

# contig_classification_results

# # eukaryotic top hit - kraken is asserting these are human contamination
# sum(contig_classification_results[!, "Eukaryota"])

# # novel sequences are second hit, this is exciting!
# sum(contig_classification_results[!, "Unclassified"])

# sum(contig_classification_results[!, "Bacteria"])

# sum(contig_classification_results[!, "Other"])

# sum(contig_classification_results[!, "Viruses"])

# sum(contig_classification_results[!, "Archaea"])

# contig_classification_results_summary = contig_classification_results[!, ["Contig", "top_kingdom", "blast_hits"]]

# contig_classification_results_summary

# StatsBase.countmap(contig_classification_results_summary[!, "blast_hits"])

# contig_classification_results_summary[contig_classification_results_summary[!, "blast_hits"] .== 5, :]